# Quick Start

## Access stellar model grid data

In [ ]:
from isochrones.mist import MISTIsochroneGrid

grid = MISTIsochroneGrid()
print(len(grid.df))
grid.df.head()  # Just the first few rows

In [ ]:
from isochrones.mist import MISTEvolutionTrackGrid

grid_tracks = MISTEvolutionTrackGrid()
print(len(grid_tracks.df))
grid_tracks.df.head()

## Interpolate stellar properites

In [ ]:
from isochrones import get_ichrone
mist = get_ichrone('mist')
eep = mist.get_eep(1.01, 9.76, 0.03, accurate=True)
mist.interp_value([eep, 9.76, 0.03], ['Teff', 'logg', 'radius', 'density'])

In [ ]:
mist.interp_mag([eep, 9.76, 0.03, 200, 0.1], bands=['G', 'BP', 'RP'])

## Generate synthetic properties of stars 

In [ ]:
from isochrones import get_ichrone
tracks = get_ichrone('mist', tracks=True)

mass, age, feh = (1.03, 9.72, -0.11)

tracks.generate(mass, age, feh, return_dict=True)  # "accurate=True" makes more accurate, but slower

In [ ]:
from isochrones.priors import ChabrierPrior
import numpy as np

# Simulate a 1000-star cluster at 8kpc

N = 1000
masses = ChabrierPrior().sample(N)
feh = -1.8
age = np.log10(6e9)  # 6 Gyr
distance = 8000.  # 8 kpc 
AV = 0.15

# By default this will return a dataframe
%timeit tracks.generate(masses, age, feh, distance=distance, AV=AV)
df = tracks.generate(masses, age, feh, distance=distance, AV=AV)

In [ ]:
df = df.dropna()
print(len(df)) # about half of the original simulated stars are nans
df.head()  

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import hvplot.pandas

df['BP-RP'] = df.BP - df.RP
df.hvplot.scatter('BP-RP', 'G', hover_cols=['mass', 'radius', 'Teff', 'logg', 'eep']).options(invert_yaxis=True, width=600)

## Fit physical parameters of a star to observed data

In [ ]:
from isochrones import get_ichrone, SingleStarModel

mist = get_ichrone('mist', bands=['BP', 'RP'])
params = {'Teff': (5700, 100), 'logg': (4.5, 0.1), 'feh': (0.0, 0.15), 
          'BP': (10.42, 0.01), 'RP': (9.54, 0.01), 
          'parallax': (10, 0.5)}  # mas
mod = SingleStarModel(mist, **params)
mod.fit()

In [ ]:
%matplotlib inline

mod.corner_physical();

Check out the numerical sampling results:

In [ ]:
mod.samples.describe()

And the derived parameters at those samples:

In [ ]:
mod.derived_samples.describe()

Eyeball your posterior predictive with:

In [ ]:
mod.corner_observed();

## Fit a binary star model

In [ ]:
from isochrones import BinaryStarModel
mod2 = BinaryStarModel(mist, **params)

In [ ]:
mod2.fit()
mod2.corner_physical();

In [ ]:
mod2.derived_samples.head()

In [ ]:
mod2.corner_observed();